In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

In [19]:
#handeling null data and droping non neccesary data columns

df = pd.read_csv("/content/Train.csv")
df['Outlet_Size'] = ['Unknown' if pd.isna(value) else value for value in df['Outlet_Size']]
df_weights = df['Item_Weight'].mean()
df['Item_Weight'].fillna(df_weights, inplace=True)
df= df.drop(labels=['Item_Identifier','Outlet_Identifier'] ,axis = 1 )




df['Item_Fat_Content']=['Low Fat'if item =="LF" else item for item in df['Item_Fat_Content']]
df['Item_Fat_Content']=['Low Fat'if item =="low fat" else item for item in df['Item_Fat_Content']]
df['Item_Fat_Content']=['Regular'if item =="reg" else item for item in df['Item_Fat_Content']]
#categorical 2 numerical
categorical_columns = ['Item_Fat_Content','Outlet_Size','Item_Type','Outlet_Location_Type','Outlet_Type']


one_hot_encoded = pd.get_dummies(df[categorical_columns], columns=categorical_columns, prefix=categorical_columns)


# Concatenate the one-hot encoded columns with the original DataFrame
df_encoded = pd.concat([df, one_hot_encoded], axis=1)

# Drop the original categorical column if desired
df_encoded = df_encoded.drop(['Item_Fat_Content','Outlet_Size','Item_Type','Outlet_Location_Type','Outlet_Type'], axis=1)

# Now df_encoded contains your data with one-hot encoded columns


#Standardization
numerical_columns = [ 'Item_Weight','Item_Visibility','Item_MRP','Outlet_Establishment_Year','Item_Outlet_Sales']
scaler = StandardScaler()

df_encoded[numerical_columns] = scaler.fit_transform(df_encoded[numerical_columns])

print(df_encoded.head())


   Item_Weight  Item_Visibility  Item_MRP  Outlet_Establishment_Year  \
0    -0.841872        -0.970732  1.747454                   0.139541   
1    -1.641706        -0.908111 -1.489023                   1.334103   
2     1.098554        -0.956917  0.010040                   0.139541   
3     1.500838        -1.281758  0.660050                   0.020085   
4    -0.929428        -1.281758 -1.399220                  -1.293934   

   Item_Outlet_Sales  Item_Fat_Content_Low Fat  Item_Fat_Content_Regular  \
0           0.910601                         1                         0   
1          -1.018440                         0                         1   
2          -0.049238                         1                         0   
3          -0.849103                         0                         1   
4          -0.695373                         1                         0   

   Outlet_Size_High  Outlet_Size_Medium  Outlet_Size_Small  ...  \
0                 0                   1    

"\nnew_df = pd.get_dummies(df)\n\n#  sending item outlet sales into the end of dataset\n\n# Replace 'column_name' with the name of the column you want to move\ncolumn_name = 'Item_Outlet_Sales'\n\n# Pop the column from its current position\ncolumn_to_move = new_df.pop(column_name)\n\n# Add the popped column back to the DataFrame at the end\nnew_df[column_name] = column_to_move\n\n\nheader_names = new_df.columns\nnew_df\n\n\n"

In [32]:
#cross val


features = df_encoded.drop('Item_Outlet_Sales',axis = 1)
target = df_encoded['Item_Outlet_Sales']


#Random forest

regressor = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores= cross_val_score(regressor , features , target ,cv = 5 ,scoring='neg_mean_squared_error')*-1
mean_cv_score = cv_scores.mean()
std_cv_score = cv_scores.std()
y_pred = cross_val_predict(regressor, features, target, cv=5)
print("Mean CV Score:", mean_cv_score)
print("Std CV Score:", std_cv_score)

squared_errors = (y_pred - target) ** 2
mse = np.mean(squared_errors)
rmse = np.sqrt(mse)
print ('the RMSE is :',rmse)

#r2
# Perform cross-validation (using 5-fold cross-validation as an example)
cv_scores = cross_val_score(regressor, features, target, cv=5, scoring='r2')

# Calculate the mean and standard deviation of the R-squared scores
mean_r2_score = cv_scores.mean()
std_r2_score = cv_scores.std()

print("Mean R-squared Score:", mean_r2_score)
print("Std R-squared Score:", std_r2_score)

Mean CV Score: 0.4461578928214391
Std CV Score: 0.0036126207087612823
the RMSE is : 0.6679509665480302
Mean R-squared Score: 0.5529732539614282
Std R-squared Score: 0.01820340941490148
